In [15]:

import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
from pyspark.sql import SparkSession
import pandas as pd
spark = SparkSession.builder.appName('BDAS_Iteration4').getOrCreate()
df=spark.read.load("./stanfordMOOCForumPostsSet.csv", inferSchema = True,format="csv",header="true")
df.printSchema()#schema details
print(df.count(),len(df.columns))



root
 |-- Opinion: integer (nullable = true)
 |-- Question: integer (nullable = true)
 |-- Answer: integer (nullable = true)
 |-- Sentiment: double (nullable = true)
 |-- Confusion: double (nullable = true)
 |-- Urgency: double (nullable = true)
 |-- CourseType: string (nullable = true)
 |-- forum_post_id: string (nullable = true)
 |-- course_display_name: string (nullable = true)
 |-- forum_uid: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- post_type: string (nullable = true)
 |-- anonymous: boolean (nullable = true)
 |-- anonymous_to_peers: boolean (nullable = true)
 |-- up_count: integer (nullable = true)
 |-- comment_thread_id: string (nullable = true)
 |-- reads: integer (nullable = true)

2999 17


In [16]:
df.createOrReplaceTempView('Data')
Results = spark.sql("SELECT reads FROM Data")
Results.show(40,False)


+-----+
|reads|
+-----+
|41   |
|55   |
|25   |
|0    |
|3    |
|3    |
|362  |
|11   |
|8    |
|20   |
|118  |
|39   |
|3    |
|27   |
|65   |
|6    |
|48   |
|6    |
|118  |
|7    |
|2    |
|7    |
|20   |
|5    |
|2    |
|17   |
|3    |
|4    |
|150  |
|10   |
|1    |
|5    |
|1117 |
|17   |
|2    |
|3    |
|12   |
|47   |
|3    |
|19   |
+-----+
only showing top 40 rows



In [17]:
df.createOrReplaceTempView('Data')
Results = spark.sql("SELECT DISTINCT up_count FROM Data")
Results.show(40,False)

+--------+
|up_count|
+--------+
|28      |
|12      |
|1       |
|13      |
|6       |
|16      |
|3       |
|20      |
|5       |
|191     |
|15      |
|37      |
|9       |
|4       |
|8       |
|7       |
|10      |
|73      |
|25      |
|24      |
|75      |
|11      |
|14      |
|2       |
|0       |
+--------+



In [18]:
df.createOrReplaceTempView('Data')
Results = spark.sql("SELECT DISTINCT Confusion FROM Data")
Results.show(40,False)
print(df.columns)

+---------+
|Confusion|
+---------+
|7.0      |
|3.5      |
|4.5      |
|6.5      |
|2.5      |
|1.0      |
|4.0      |
|3.0      |
|2.0      |
|1.5      |
|6.0      |
|5.0      |
|5.5      |
+---------+

['Opinion', 'Question', 'Answer', 'Sentiment', 'Confusion', 'Urgency', 'CourseType', 'forum_post_id', 'course_display_name', 'forum_uid', 'created_at', 'post_type', 'anonymous', 'anonymous_to_peers', 'up_count', 'comment_thread_id', 'reads']


In [19]:
newdf=df;
df=df.drop('anonymous','anonymous_to_peers')
print(df.columns)

['Opinion', 'Question', 'Answer', 'Sentiment', 'Confusion', 'Urgency', 'CourseType', 'forum_post_id', 'course_display_name', 'forum_uid', 'created_at', 'post_type', 'up_count', 'comment_thread_id', 'reads']


In [20]:
from pyspark.sql.functions import isnan, when, count, col
df.describe().filter(col("summary") == "count").show()
df.select([count(when(isnan(c) | col(c).isNull(),c)).alias(c) for c in df.columns]).show()

+-------+-------+--------+------+---------+---------+-------+----------+-------------+-------------------+---------+----------+---------+--------+-----------------+-----+
|summary|Opinion|Question|Answer|Sentiment|Confusion|Urgency|CourseType|forum_post_id|course_display_name|forum_uid|created_at|post_type|up_count|comment_thread_id|reads|
+-------+-------+--------+------+---------+---------+-------+----------+-------------+-------------------+---------+----------+---------+--------+-----------------+-----+
|  count|   2999|    2999|  2999|     2999|     2999|   2999|      2999|         2999|               2999|     2999|      2999|     2999|    2999|             2999| 2999|
+-------+-------+--------+------+---------+---------+-------+----------+-------------+-------------------+---------+----------+---------+--------+-----------------+-----+

+-------+--------+------+---------+---------+-------+----------+-------------+-------------------+---------+----------+---------+--------+------

In [21]:
df.describe().show() 

+-------+-------------------+-------------------+-------------------+------------------+-----------------+------------------+----------+--------------------+--------------------+--------------------+----------------+-------------+------------------+--------------------+------------------+
|summary|            Opinion|           Question|             Answer|         Sentiment|        Confusion|           Urgency|CourseType|       forum_post_id| course_display_name|           forum_uid|      created_at|    post_type|          up_count|   comment_thread_id|             reads|
+-------+-------------------+-------------------+-------------------+------------------+-----------------+------------------+----------+--------------------+--------------------+--------------------+----------------+-------------+------------------+--------------------+------------------+
|  count|               2999|               2999|               2999|              2999|             2999|              2999|     

In [22]:
df.select('reads').show()

+-----+
|reads|
+-----+
|   41|
|   55|
|   25|
|    0|
|    3|
|    3|
|  362|
|   11|
|    8|
|   20|
|  118|
|   39|
|    3|
|   27|
|   65|
|    6|
|   48|
|    6|
|  118|
|    7|
+-----+
only showing top 20 rows



In [23]:
newdf=newdf.drop('forum_post_id','forum_uid')
print(newdf.columns)

['Opinion', 'Question', 'Answer', 'Sentiment', 'Confusion', 'Urgency', 'CourseType', 'course_display_name', 'created_at', 'post_type', 'anonymous', 'anonymous_to_peers', 'up_count', 'comment_thread_id', 'reads']


In [24]:
newdf.printSchema()#schema details

root
 |-- Opinion: integer (nullable = true)
 |-- Question: integer (nullable = true)
 |-- Answer: integer (nullable = true)
 |-- Sentiment: double (nullable = true)
 |-- Confusion: double (nullable = true)
 |-- Urgency: double (nullable = true)
 |-- CourseType: string (nullable = true)
 |-- course_display_name: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- post_type: string (nullable = true)
 |-- anonymous: boolean (nullable = true)
 |-- anonymous_to_peers: boolean (nullable = true)
 |-- up_count: integer (nullable = true)
 |-- comment_thread_id: string (nullable = true)
 |-- reads: integer (nullable = true)



In [25]:
#Dropping any Null and missing values
df.printSchema()
df.dropna()
df.select([count(when(isnan(c) | col(c).isNull(),c)).alias(c) for c in df.columns]).show()

root
 |-- Opinion: integer (nullable = true)
 |-- Question: integer (nullable = true)
 |-- Answer: integer (nullable = true)
 |-- Sentiment: double (nullable = true)
 |-- Confusion: double (nullable = true)
 |-- Urgency: double (nullable = true)
 |-- CourseType: string (nullable = true)
 |-- forum_post_id: string (nullable = true)
 |-- course_display_name: string (nullable = true)
 |-- forum_uid: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- post_type: string (nullable = true)
 |-- up_count: integer (nullable = true)
 |-- comment_thread_id: string (nullable = true)
 |-- reads: integer (nullable = true)

+-------+--------+------+---------+---------+-------+----------+-------------+-------------------+---------+----------+---------+--------+-----------------+-----+
|Opinion|Question|Answer|Sentiment|Confusion|Urgency|CourseType|forum_post_id|course_display_name|forum_uid|created_at|post_type|up_count|comment_thread_id|reads|
+-------+--------+------+---------+--

In [26]:
for col in newdf.columns:
    print(col,"\t","with null values: ",newdf.filter(newdf[col].isNull()).count())

Opinion 	 with null values:  0
Question 	 with null values:  0
Answer 	 with null values:  0
Sentiment 	 with null values:  0
Confusion 	 with null values:  0
Urgency 	 with null values:  0
CourseType 	 with null values:  0
course_display_name 	 with null values:  0
created_at 	 with null values:  0
post_type 	 with null values:  0
anonymous 	 with null values:  0
anonymous_to_peers 	 with null values:  0
up_count 	 with null values:  0
comment_thread_id 	 with null values:  0
reads 	 with null values:  0


In [27]:
#Data Cleaning for Urgency
import pyspark.sql.functions as fun
newdf = newdf.withColumn(
"Urgency",
    fun.when(
        (fun.col("Urgency") >1)&
        (fun.col("Urgency") <=7), fun.col("Urgency")).otherwise(fun.lit(None))
)
na_df=newdf.dropna()
na_df.select('Urgency').distinct().show(1000,False)

+-------+
|Urgency|
+-------+
|7.0    |
|3.5    |
|4.5    |
|6.5    |
|2.5    |
|4.0    |
|3.0    |
|2.0    |
|1.5    |
|6.0    |
|5.0    |
|5.5    |
+-------+



In [28]:
#Data Cleaning for Sentiment
import pyspark.sql.functions as fun
newdf = newdf.withColumn(
"Sentiment",
    fun.when(
        (fun.col("Sentiment") >1)&
        (fun.col("Sentiment") <=7), fun.col("Sentiment")).otherwise(fun.lit(None))
)
na_df=newdf.dropna()
na_df.select('Sentiment').distinct().show(1000,False)

+---------+
|Sentiment|
+---------+
|7.0      |
|3.5      |
|4.5      |
|6.5      |
|2.5      |
|4.0      |
|3.0      |
|2.0      |
|1.5      |
|6.0      |
|5.0      |
|5.5      |
+---------+



In [29]:
na_df.createOrReplaceTempView('data')
Results = spark.sql("SELECT min(Urgency), max(Urgency) FROM data")
Results.show(100,False)


+------------+------------+
|min(Urgency)|max(Urgency)|
+------------+------------+
|1.5         |7.0         |
+------------+------------+



In [30]:
na_df.createOrReplaceTempView('data')
Results = spark.sql("SELECT min(Confusion), max(Confusion) FROM data")
Results.show(100,False)


+--------------+--------------+
|min(Confusion)|max(Confusion)|
+--------------+--------------+
|1.0           |6.5           |
+--------------+--------------+



In [31]:
na_df.createOrReplaceTempView('data')
Results = spark.sql("SELECT min(Sentiment), max(Sentiment) FROM data")
Results.show(100,False)


+--------------+--------------+
|min(Sentiment)|max(Sentiment)|
+--------------+--------------+
|1.5           |7.0           |
+--------------+--------------+



In [32]:
# Deriving New Column
df1 = na_df.withColumn('Participation',(fun.col('Opinion') == 1 ) | (fun.col('Question') == 1) | (fun.col('Answer') == 1))
df1.select('Participation').show()
df1.printSchema()#schema details

+-------------+
|Participation|
+-------------+
|         true|
|         true|
|         true|
|         true|
|         true|
|         true|
|         true|
|         true|
|         true|
|         true|
|         true|
|         true|
|         true|
|         true|
|         true|
|         true|
|         true|
|         true|
|         true|
|         true|
+-------------+
only showing top 20 rows

root
 |-- Opinion: integer (nullable = true)
 |-- Question: integer (nullable = true)
 |-- Answer: integer (nullable = true)
 |-- Sentiment: double (nullable = true)
 |-- Confusion: double (nullable = true)
 |-- Urgency: double (nullable = true)
 |-- CourseType: string (nullable = true)
 |-- course_display_name: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- post_type: string (nullable = true)
 |-- anonymous: boolean (nullable = true)
 |-- anonymous_to_peers: boolean (nullable = true)
 |-- up_count: integer (nullable = true)
 |-- comment_thread_id: string (nu

In [33]:
#Merging the data
data2 = spark.read.load("./stanfordMOOCForumPostsSet2.csv",format="csv", inferSchema = True, header="true")
newdf2 = data2.drop('forum_post_id','forum_uid')
import pyspark.sql.functions as fun
newdf2 = newdf2.withColumn(
"Urgency",
    fun.when(
        (fun.col("Urgency") >1)&
        (fun.col("Urgency") <=7), fun.col("Urgency")).otherwise(fun.lit(None))
)
newdf2 = newdf2.withColumn(
"Sentiment",
    fun.when(
        (fun.col("Sentiment") >1)&
        (fun.col("Sentiment") <=7), fun.col("Sentiment")).otherwise(fun.lit(None))
)
newdf2 = newdf2.withColumn(
"Confusion",
    fun.when(
        (fun.col("Confusion") >1)&
        (fun.col("Confusion") <=7), fun.col("Confusion")).otherwise(fun.lit(None))
)
na_df2 = newdf2.dropna()
#print(na_df2.count(),len(na_df2.columns))
df2 = na_df2.withColumn('Participation',(fun.col('Opinion') == 1 ) | (fun.col('Question') == 1) | (fun.col('Answer') == 1))
print(df1.count(),len(df1.columns))
print(df2.count(),len(df2.columns))
#Merging
finaldf = df1.union(df2)
print(finaldf.count(),len(finaldf.columns))


2878 16
993 16
3871 16


In [34]:
#Converting string value to integer 
from pyspark.ml.feature import StringIndexer
encodeddf = finaldf
x = finaldf.columns
x.remove("anonymous") #Its already boolean
x.remove("anonymous_to_peers") #Its already boolean
x.remove("Participation") #Its already boolean
for i in x:
    ic = i
    oc = i+" encoded" 
    indexer = StringIndexer(inputCol=ic, outputCol=oc)
    encodeddf = indexer.fit(encodeddf).transform(encodeddf)

In [35]:
y = [i for i in encodeddf.columns if i not in x] #storing only those columns which are numerical
encodeddf = encodeddf[y]

In [36]:
encodeddf.show(5)

+---------+------------------+-------------+---------------+----------------+--------------+-----------------+-----------------+---------------+------------------+---------------------------+------------------+-----------------+----------------+-------------------------+-------------+
|anonymous|anonymous_to_peers|Participation|Opinion encoded|Question encoded|Answer encoded|Sentiment encoded|Confusion encoded|Urgency encoded|CourseType encoded|course_display_name encoded|created_at encoded|post_type encoded|up_count encoded|comment_thread_id encoded|reads encoded|
+---------+------------------+-------------+---------------+----------------+--------------+-----------------+-----------------+---------------+------------------+---------------------------+------------------+-----------------+----------------+-------------------------+-------------+
|    false|             false|         true|            0.0|             0.0|           0.0|              7.0|              7.0|            1.

In [37]:
encodeddf.printSchema()
df=encodeddf;

root
 |-- anonymous: boolean (nullable = true)
 |-- anonymous_to_peers: boolean (nullable = true)
 |-- Participation: boolean (nullable = true)
 |-- Opinion encoded: double (nullable = true)
 |-- Question encoded: double (nullable = true)
 |-- Answer encoded: double (nullable = true)
 |-- Sentiment encoded: double (nullable = true)
 |-- Confusion encoded: double (nullable = true)
 |-- Urgency encoded: double (nullable = true)
 |-- CourseType encoded: double (nullable = true)
 |-- course_display_name encoded: double (nullable = true)
 |-- created_at encoded: double (nullable = true)
 |-- post_type encoded: double (nullable = true)
 |-- up_count encoded: double (nullable = true)
 |-- comment_thread_id encoded: double (nullable = true)
 |-- reads encoded: double (nullable = true)



In [38]:
from pyspark.sql.functions import col

replacements = {c:c.replace(' encoded','') for c in df.columns }

df = df.select([col(c).alias(replacements.get(c, c)) for c in df.columns])

In [39]:
#4.1 Data deduction - correlation matrix with heatmap 
from pyspark.mllib.stat import Statistics
from pyspark import SparkContext, SparkConf
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
conf = SparkConf().setAppName('BDAS_Iteration4').setMaster("local[*]")
sc = SparkContext.getOrCreate(conf)

resultRDD = df.rdd.map(lambda row: row[0:])
corrlation_matrix = Statistics.corr(resultRDD, method = 'pearson')
rdd0 = sc.parallelize(corrlation_matrix)
rdd1 = rdd0.map(lambda x: [float(i) for i in x])
corrlation_matrix[1]

plt.figure(figsize=(22,10))
sns.heatmap(corrlation_matrix, cmap="Purples", annot = True)
plt.show()

<Figure size 2200x1000 with 2 Axes>

In [40]:
#Convert columns and transform them to numerical
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer


column_vec_in = [
                'Opinion',
                'Question',
                'Answer',
                'Sentiment',
                'Confusion',
                'Urgency',
                'course_display_name',
                'created_at',
                'post_type',
                'up_count',
                'comment_thread_id',
                'reads'
                ]

column_vec_out = ['Opinion_Vec',
                'Question_Vec',
                'Answer_Vec',
                'Sentiment_Vec',
                'Confusion_Vec',
                'Urgency_Vec',
                'course_display_name_Vec',
                'created_at_Vec',
                'post_type_Vec',
                'up_count_Vec',
                'comment_thread_id_Vec',
                'reads_Vec']

indexers = [StringIndexer(inputCol = x, outputCol=x + "_tmp") for x in column_vec_in]

encoders = [OneHotEncoder(dropLast=False, inputCol = x + "_tmp", outputCol = y)
           for x, y in zip(column_vec_in, column_vec_out)]

tmp = [[i, j] for i, j in zip(indexers, encoders)]
tmp = [i for sublist in tmp for i in sublist]

In [41]:
#combining atttributes into the of features and target is Coursetype
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

cols_now = ['Opinion_Vec',
                'Question_Vec',
                'Answer_Vec',
                'Sentiment_Vec',
                'Confusion_Vec',
                'Urgency_Vec',
                'course_display_name_Vec',
                'created_at_Vec',
                'post_type_Vec',
                'up_count_Vec',
                'comment_thread_id_Vec',
                'reads_Vec']

assembler_features = VectorAssembler(inputCols = cols_now, outputCol= "features")
labelIndexer = StringIndexer(inputCol = 'CourseType', outputCol= "label")
tmp += [assembler_features, labelIndexer]
pipeline = Pipeline(stages = tmp)

In [42]:
#Training Data split 70% and 30%
allData = pipeline.fit(df).transform(df)
allData.cache()

trainingData, testData = allData.randomSplit([0.7, 0.3], seed=0)
print("Distribution of Positive and Negative in trainingData is: ", trainingData.groupBy("label").count().take(3))

Distribution of Positive and Negative in trainingData is:  [Row(label=0.0, count=1395), Row(label=1.0, count=695), Row(label=2.0, count=650)]


In [43]:
#Random Forest Classifer
from pyspark.ml.classification import RandomForestClassifier as RF
rf = RF(labelCol = "label", featuresCol = "features", numTrees = 200, maxDepth = 3)
fit = rf.fit(trainingData)
transformed = fit.transform(testData)

fit.featureImportances

SparseVector(5313, {0: 0.0458, 1: 0.0603, 2: 0.0089, 3: 0.0162, 4: 0.0102, 5: 0.0358, 6: 0.04, 7: 0.0164, 8: 0.0009, 9: 0.0024, 10: 0.0072, 12: 0.0049, 13: 0.006, 14: 0.0, 15: 0.0011, 18: 0.0244, 19: 0.0219, 20: 0.0065, 21: 0.0063, 22: 0.0091, 23: 0.0073, 24: 0.0034, 25: 0.0001, 26: 0.0017, 30: 0.0167, 31: 0.0121, 32: 0.01, 34: 0.0002, 35: 0.012, 36: 0.0033, 37: 0.0157, 38: 0.0016, 39: 0.009, 40: 0.0031, 41: 0.0002, 42: 0.041, 43: 0.0689, 44: 0.0293, 45: 0.0132, 46: 0.02, 47: 0.001, 50: 0.0012, 60: 0.0, 68: 0.0011, 77: 0.0, 81: 0.0002, 83: 0.0003, 92: 0.0001, 114: 0.0012, 126: 0.0007, 140: 0.0017, 146: 0.0003, 147: 0.0001, 153: 0.0001, 159: 0.0, 184: 0.0, 193: 0.0001, 231: 0.0, 232: 0.0002, 302: 0.0, 355: 0.0001, 361: 0.0, 371: 0.0006, 381: 0.0006, 395: 0.0001, 420: 0.0, 428: 0.0, 457: 0.0004, 486: 0.0, 540: 0.0001, 541: 0.0, 575: 0.0006, 576: 0.0, 609: 0.0002, 626: 0.001, 677: 0.0, 690: 0.0, 728: 0.0003, 798: 0.0007, 804: 0.0002, 815: 0.0, 818: 0.0001, 828: 0.0, 845: 0.0, 891: 0.0002,

In [44]:
#6.3.2 Random Forest
from pyspark.mllib.evaluation import BinaryClassificationMetrics as metric
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName('BDAS_Iteration4').setMaster("local[*]")
sc = SparkContext.getOrCreate(conf)

results = transformed.select(['probability', 'label'])

results_collect = results.collect()
results_list = [(float(i[0][0]), 1.0-float(i[1])) for i in results_collect]

scoreAndLabels = sc.parallelize(results_list)
metrics = metric(scoreAndLabels)
print("The ROC score is: ", metrics.areaUnderROC)
# Area under precision-recall curve
print("Area under PR = %s" % metrics.areaUnderPR)

from sklearn.metrics import classification_report, confusion_matrix
y_true = transformed.select(['label']).collect()
y_pred = transformed.select(['prediction']).collect()
print(classification_report(y_true, y_pred))

The ROC score is:  0.9959700181898014
Area under PR = 0.9967183647218204
              precision    recall  f1-score   support

         0.0       0.56      1.00      0.72       596
         1.0       1.00      0.01      0.02       271
         2.0       1.00      0.24      0.39       264

    accuracy                           0.59      1131
   macro avg       0.85      0.42      0.37      1131
weighted avg       0.77      0.59      0.47      1131



In [45]:
#decision tree
from pyspark.ml.classification import DecisionTreeClassifier as DT
tree = DT(labelCol = "label", featuresCol = "features", maxDepth = 3)
tree_fit = tree.fit(trainingData)
tree_transformed = tree_fit.transform(testData)

tree_fit.featureImportances

SparseVector(5313, {42: 0.6047, 43: 0.3953})

In [46]:
#Decision Tree
from pyspark.mllib.evaluation import BinaryClassificationMetrics as metric
from pyspark import SparkContext, SparkConf
from sklearn.metrics import classification_report, confusion_matrix
conf = SparkConf().setAppName('BDAS_Iteration4').setMaster("local[*]")
sc = SparkContext.getOrCreate(conf)

tree_results = tree_transformed.select(['probability', 'label'])

tree_results_collect = tree_results.collect()
tree_results_list = [(float(i[0][0]), 1.0-float(i[1])) for i in tree_results_collect]

tree_scoreAndLabels = sc.parallelize(tree_results_list)
tree_metrics = metric(tree_scoreAndLabels)
print("The ROC score is: ", tree_metrics.areaUnderROC)
# Area under precision-recall curve
print("Area under PR = %s" % tree_metrics.areaUnderPR)


y_true = tree_transformed.select(['label']).collect()
y_pred = tree_transformed.select(['prediction']).collect()
print(classification_report(y_true, y_pred))

The ROC score is:  1.0
Area under PR = 1.0
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       596
         1.0       1.00      1.00      1.00       271
         2.0       1.00      1.00      1.00       264

    accuracy                           1.00      1131
   macro avg       1.00      1.00      1.00      1131
weighted avg       1.00      1.00      1.00      1131



In [47]:

#7 Data Mining
from pyspark.ml.classification import DecisionTreeClassifier,RandomForestClassifier
from pyspark.ml import Pipeline

dtc = DecisionTreeClassifier(labelCol='label',featuresCol='features')
rfc = RandomForestClassifier(labelCol='label',featuresCol='features')

dtc_model = dtc.fit(trainingData)
rfc_model = rfc.fit(trainingData)


dtc_predictions = dtc_model.transform(testData)
rfc_predictions = rfc_model.transform(testData)


from pyspark.ml.evaluation import BinaryClassificationEvaluator

my_binary_eval = BinaryClassificationEvaluator(labelCol = 'label')
print("Decision Tree Classifier")
print(my_binary_eval.evaluate(dtc_predictions))

print("Random Forest Classifer")
print(my_binary_eval.evaluate(rfc_predictions))



Decision Tree Classifier
0.7532710280373831
Random Forest Classifer
0.8606269209057278


In [48]:
# Data Mining
#Evaluating Accuracy for all test model
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

dtc_acc = acc_evaluator.evaluate(dtc_predictions)
rfc_acc = acc_evaluator.evaluate(rfc_predictions)

print("Here are the results!")
print('-'*40)
print('A single decision tree has an accuracy of: {0:2.2f}%'.format(dtc_acc*100))
print('-'*40)
print('A random forest ensemble has an accuracy of: {0:2.2f}%'.format(rfc_acc*100))



Here are the results!
----------------------------------------
A single decision tree has an accuracy of: 100.00%
----------------------------------------
A random forest ensemble has an accuracy of: 63.22%


In [49]:

#Decision Tree Features Important
from pyspark.ml.regression import DecisionTreeRegressor as DTR
dtr_tree = DTR(labelCol = "label", featuresCol = "features", maxDepth = 3)
dtr_tree_fit = dtr_tree.fit(trainingData)
dtr_tree_transformed = dtr_tree_fit.transform(testData)

dtr_tree_fit.featureImportances

SparseVector(5313, {42: 0.8177, 43: 0.1823})

In [37]:
from pyspark.mllib.evaluation import RegressionMetrics as metric
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName('BDAS_Iteration4').setMaster("local[*]")
sc = SparkContext.getOrCreate(conf)

dtr_results = dtr_tree_transformed.select(['features', 'label'])

dtr_results_collect = dtr_results.collect()
dtr_results_list = [(float(i[0][0]), 1.0-float(i[1])) for i in dtr_results_collect]

dtr_scoreAndLabels = sc.parallelize(dtr_results_list)
dtr_metrics = metric(dtr_scoreAndLabels)
print("Mean Absolute Error is: ", dtr_metrics.meanAbsoluteError)
print("Mean Squared Error: ", dtr_metrics.meanSquaredError)
print("Root Mean Squared Error is: ", dtr_metrics.rootMeanSquaredError)
print("R2 is: ", dtr_metrics.r2)
print("Explained Variance is: ", dtr_metrics.explainedVariance)

Mean Absolute Error is:  0.376657824933687
Mean Squared Error:  0.4792219274977896
Root Mean Squared Error is:  0.6922585698261811
R2 is:  0.2892203015644059
Explained Variance is:  0.34683827915328475


In [50]:
#Random Forest Feature Importances
from pyspark.ml.regression import RandomForestRegressor as RFR
rfr_tree = RFR(labelCol = "label", featuresCol = "features", maxDepth = 3)
rfr_tree_fit = rfr_tree.fit(trainingData)
rfr_tree_transformed = rfr_tree_fit.transform(testData)

rfr_tree_fit.featureImportances

SparseVector(5313, {0: 0.1046, 1: 0.0516, 4: 0.0156, 5: 0.0058, 6: 0.0061, 18: 0.0003, 19: 0.0125, 25: 0.0002, 30: 0.0007, 36: 0.0003, 42: 0.549, 43: 0.1995, 44: 0.0471, 45: 0.0007, 46: 0.0009, 3828: 0.001, 3829: 0.0009, 3830: 0.0006, 5038: 0.002, 5053: 0.0005})

In [51]:
from pyspark.mllib.evaluation import RegressionMetrics as metric
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName('BDAS_Iteration4').setMaster("local[*]")
sc = SparkContext.getOrCreate(conf)

rfr_results = rfr_tree_transformed.select(['features', 'label'])

rfr_results_collect = rfr_results.collect()
rfr_results_list = [(float(i[0][0]), 1.0-float(i[1])) for i in rfr_results_collect]

rfr_scoreAndLabels = sc.parallelize(rfr_results_list)
rfr_metrics = metric(rfr_scoreAndLabels)
print("Mean Absolute Error is: ", rfr_metrics.meanAbsoluteError)
print("Mean Squared Error: ", rfr_metrics.meanSquaredError)
print("Root Mean Squared Error is: ", rfr_metrics.rootMeanSquaredError)
print("R2 is: ", rfr_metrics.r2)
print("Explained Variance is: ", rfr_metrics.explainedVariance)

Mean Absolute Error is:  0.376657824933687
Mean Squared Error:  0.4792219274977896
Root Mean Squared Error is:  0.6922585698261811
R2 is:  0.2892203015644059
Explained Variance is:  0.34683827915328475


In [38]:
from pyspark.ml.feature import ChiSqSelector as CSS
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName('BDAS_Iteration4').setMaster("local[*]")
sc = SparkContext.getOrCreate(conf)

selector = CSS(numTopFeatures=10, featuresCol="features",
                         outputCol="selectedFeatures", labelCol="label")
selector_result = selector.fit(allData)
selector_transformed = selector_result.transform(allData)

print("ChiSqSelector output with top %d features selected" % selector.getNumTopFeatures())
selector_transformed.show(10)

ChiSqSelector output with top 10 features selected
+---------+------------------+-------------+-------+--------+------+---------+---------+-------+----------+-------------------+----------+---------+--------+-----------------+-----+-----------+-------------+------------+-------------+----------+-------------+-------------+--------------+-------------+---------------+-----------+--------------+-----------------------+-----------------------+--------------+-------------------+-------------+-------------+------------+--------------+---------------------+---------------------+---------+-----------------+--------------------+-----+--------------------+
|anonymous|anonymous_to_peers|Participation|Opinion|Question|Answer|Sentiment|Confusion|Urgency|CourseType|course_display_name|created_at|post_type|up_count|comment_thread_id|reads|Opinion_tmp|  Opinion_Vec|Question_tmp| Question_Vec|Answer_tmp|   Answer_Vec|Sentiment_tmp| Sentiment_Vec|Confusion_tmp|  Confusion_Vec|Urgency_tmp|   Urgency_Vec

In [ ]:
#Decision Tree Features Important
from pyspark.ml.regression import DecisionTreeRegressor as DTR
import matplotlib.pyplot as plt
dtr_tree = DTR(labelCol = "label", featuresCol = "features", maxDepth = 3)
dtr_tree_fit = dtr_tree.fit(trainingData)
dtr_tree_transformed = dtr_tree_fit.transform(testData)

dtr_tree_fit.featureImportances

plt.figure(figsize=(23,6))
plt.bar("label", dtr_tree_fit.featureImportances)
plt.xticks(rotation=90)
plt.title('Decision Tree Feature Importance')
plt.show(5)


In [ ]:
#Random Forest Feature Importances
from pyspark.ml.regression import RandomForestRegressor as RFR
import matplotlib.pyplot as plt
rfr_tree = RFR(labelCol = "label", featuresCol = "features", maxDepth = 3)
rfr_tree_fit = rfr_tree.fit(trainingData)
rfr_tree_transformed = rfr_tree_fit.transform(testData)

rfr_tree_fit.featureImportances

plt.figure(figsize=(23,6))
plt.bar("label", rfr_tree_fit.featureImportances)
plt.xticks(rotation=90)
plt.title('Random Forest Feature Importance')
plt.show(5)


In [ ]:
feature_name = [ 'Opinion',
                'Question',
                'Answer',
                'Sentiment',
                'Confusion',
                'Urgency',
                'course_display_name',
                'created_at',
                'post_type',
                'up_count',
                'comment_thread_id',
                'reads',
               'CourseType']

print(dtc_model.toDebugString)

index = 0 
for feature in feature_name:
    print(' feature ' + str(index) + " : " + feature_name[index])
    index += 1
